In [1]:
import networkx as nx
import numpy as np

In [41]:
## Returns True if the arrays representing the two partitions are adjacent by a swap and false otherwise
def is_adjacent(part1, part2):
    values = set(part1)
    parts1 = np.array(list(map(lambda v: np.ma.masked_equal(part1, v).mask, values)))
    parts2 = np.array(list(map(lambda v: np.ma.masked_equal(part2, v).mask, values)))
    
    num_diff_1 = 0
    num_diff_2 = 0
    diff_part1 = []
    diff_part2 = []
    
    is_in = lambda row, matrix: np.equal(row, parts2).all(axis=1).any()
    
    for i in range(parts1.shape[0]):
        if not np.equal(parts1[i], parts2).all(axis=1).any():
            num_diff_1 += 1
            diff_part1.append(parts1[i])
        if not np.equal(parts2[i], parts1).all(axis=1).any():
            num_diff_2 += 1
            diff_part2.append(parts2[i])
            
    if num_diff_1 != 2 and num_diff_2 != 2:
        return False
    else:
        pass
            

In [42]:
def adjacent_nodes(nodes):
    edges = []
    for i in nodes.keys():
        for j in nodes.keys():
            if i < j and is_adjacent(nodes[i], nodes[j]):
                edges.append((i, j))
    return edges

In [6]:
with open("enums/enum_[3,3]_[3]_3_rc.txt") as file:
    nodes={}
    for i, line in enumerate(file):
        nodes[i] = np.fromstring(line, dtype=int, sep=',')
    

In [7]:
nodes

{0: array([1, 2, 2, 1, 2, 3, 1, 3, 3]),
 1: array([1, 2, 3, 1, 2, 3, 1, 2, 3]),
 2: array([1, 2, 2, 1, 3, 2, 1, 3, 3]),
 3: array([1, 2, 2, 1, 1, 2, 3, 3, 3]),
 4: array([1, 1, 2, 1, 2, 2, 3, 3, 3]),
 5: array([1, 1, 2, 1, 3, 2, 3, 3, 2]),
 6: array([1, 1, 1, 2, 3, 3, 2, 2, 3]),
 7: array([1, 1, 1, 2, 2, 3, 2, 3, 3]),
 8: array([1, 1, 1, 2, 2, 2, 3, 3, 3]),
 9: array([1, 1, 2, 3, 1, 2, 3, 3, 2])}

In [43]:
adjacent_nodes(nodes)

[]

In [50]:
is_adjacent(nodes[0], nodes[8])

False

In [13]:
a = nodes[0]
a

array([1, 2, 2, 1, 2, 3, 1, 3, 3])

In [14]:
d1 = np.ma.masked_equal(a, 1).mask
d2 = np.ma.masked_equal(a, 2).mask
d3 = np.ma.masked_equal(a, 3).mask

In [34]:
np.array([d2, d3]).shape

(2, 9)

In [30]:
np.equal(d1, np.array([d1,d2, d3])).all(axis=1).any()

True